In [1]:
import importlib, sys
# script 경로 window용
sys.path.append(r"C:\Users\bong2\OneDrive\Python_Codes\00_python_scripts") 
from glob import glob
import pandas as pd
import conn_db
import helper, trade_cleaner

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 100)  

In [26]:
# 공통사항
trade_types = ['수출','수입']
unit_types = ['백만', '억']
unit_dict = {'백만': 1000,
              '억': 100000}

In [2]:
all_files = glob(conn_db.get_path('품목별_수출입_raw') + "*.xls")
df = pd.concat([pd.read_excel(file, skiprows=4)
                for file in all_files], axis=0).drop_duplicates()

C:\Users\bong2\Anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:221: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [3]:
df.head(3)

,기간,품목명,품목코드,수출중량,수입중량,수출금액,수입금액,무역수지
0,총계,NaN,NaN,"415,797,922.8","1,472,979,282.3","678,994,625","632,531,649","46,462,976"
1,2000.01,살아 있는 동물,1.0,2.0,70.7,93,743,-650
2,2000.01,육과 식용 설육(屑肉),2.0,"7,814.5","37,348.3","26,679","84,296","-57,617"


In [4]:
filt = df['기간']!='총계'
df = df.loc[filt]
names = {'기간': '날짜', 
        '품목명': '대분류',
        '수출중량': '수출중량 (ton)',
        '수입중량': '수입중량 (ton)',
        '수출금액': '수출금액 (천$)',
        '수입금액': '수입금액 (천$)',
        '무역수지': '무역수지 (천$)'}
df = df.rename(columns=names).drop(columns='품목코드')
df['날짜'] = pd.to_datetime(df['날짜'])

In [5]:
matcher = ['수출', '수입', '무역']
all_cols = df.columns.tolist()
value_cols = [col for col in all_cols if any(prcnt in col for prcnt in matcher)]

for col in value_cols:
    df[col] = pd.to_numeric(df[col].str.replace(',', ''))

In [6]:
df.head(3)

,날짜,대분류,수출중량 (ton),수입중량 (ton),수출금액 (천$),수입금액 (천$),무역수지 (천$)
1,2000-01-01,살아 있는 동물,2.0,70.7,93,743,-650
2,2000-01-01,육과 식용 설육(屑肉),7814.5,37348.3,26679,84296,-57617
3,2000-01-01,어류ㆍ갑각류ㆍ연체동물과 그 밖의 수생(水生) 무척추동물,31743.9,62955.3,91931,121804,-29873


In [7]:
df = trade_cleaner.add_units(df)

In [8]:
df.head(3)

,날짜,대분류,수출중량 (ton),수입중량 (ton),수출금액 (천$),수입금액 (천$),무역수지 (천$),수출금액 (백만$),수입금액 (백만$),무역규모 (백만$),무역수지 (백만$),수출금액 (억$),수입금액 (억$),무역규모 (억$),무역수지 (억$),무역규모 (천$),수출ton당 (천$/ton),수입ton당 (천$/ton)
1,2000-01-01,살아 있는 동물,2.0,70.7,93,743,-650,0.093,0.743,0.836,-0.650,0.00093,0.00743,0.00836,-0.00650,836,46.500000,10.509194
2,2000-01-01,육과 식용 설육(屑肉),7814.5,37348.3,26679,84296,-57617,26.679,84.296,110.975,-57.617,0.26679,0.84296,1.10975,-0.57617,110975,3.414038,2.257024
3,2000-01-01,어류ㆍ갑각류ㆍ연체동물과 그 밖의 수생(水生) 무척추동물,31743.9,62955.3,91931,121804,-29873,91.931,121.804,213.735,-29.873,0.91931,1.21804,2.13735,-0.29873,213735,2.896021,1.934770


In [22]:
df.columns.tolist()

['날짜',
 '대분류',
 '수출중량 (ton)',
 '수입중량 (ton)',
 '수출금액 (천$)',
 '수입금액 (천$)',
 '무역수지 (천$)',
 '수출금액 (백만$)',
 '수입금액 (백만$)',
 '무역규모 (백만$)',
 '무역수지 (백만$)',
 '수출금액 (억$)',
 '수입금액 (억$)',
 '무역규모 (억$)',
 '무역수지 (억$)',
 '무역규모 (천$)',
 '수출ton당 (천$/ton)',
 '수입ton당 (천$/ton)']

In [9]:
cols_long = ['날짜','대분류','수출중량 (ton)','수입중량 (ton)',
             '수출금액 (천$)','수입금액 (천$)', '무역규모 (천$)','무역수지 (천$)',
             '수출금액 (백만$)','수입금액 (백만$)','무역규모 (백만$)','무역수지 (백만$)',
             '수출금액 (억$)', '수입금액 (억$)', '무역규모 (억$)', '무역수지 (억$)',
             '수출ton당 (천$/ton)', '수입ton당 (천$/ton)']

cols_short = ['날짜','대분류','수출중량 (ton)','수입중량 (ton)',
             '수출금액 (천$)','수입금액 (천$)', '무역규모 (천$)','무역수지 (천$)',
             '수출ton당 (천$/ton)', '수입ton당 (천$/ton)']

In [10]:
# 저장
conn_db.export_(df[cols_long], '수출입_품목별')
conn_db.to_(df[cols_short], 'DB_수출입_품목별', 'import')

C:\Users\bong2\OneDrive\Python_Codes\00_python_scripts\conn_db.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(axis=1, how='all', inplace=True)
C:\Users\bong2\Anaconda3\lib\site-packages\pygsheets\worksheet.py:1366: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].astype('unicode').replace('<NA>', nan)
